# Supervised Training a Custom Encoder on the DAGHAR Dataset


In this notebook, we will train a custom encoder from scratch on the DAGHAR KuHAR dataset. The encoder can be **any** `torch.nn.Module` 

To do this, we will first define your model by:
1. Define the encoder. We going to call this encoder as `backbone`.
2. Calculating the embedding size of the `backbone`. This is necessary to define the head of the model. This is done by passing a dummy input through the `backbone` and checking the output shape.
3. Defining the head of the model. We use a standard MLP classifier as the head, whose input size is the embedding size, followed by a hidden layer of 128 units and an output layer with the number of classes in the dataset, that is, 6.
4. Creating a `SimpleSupervisedModel` with the `backbone` and the head. The `SimpleSupervisedModel` is a PyTorch Lightning module that receives the `backbone` and the head as arguments and trains the model end-to-end. This model simply forwards the input through the `backbone`, flattens the output, and passes it through the head to get the logits, whose loss is calculated using the cross-entropy loss.

Note that, for sake of reproducibility, you should only change parts related to loading the encoder and defining the head of the model. The rest of the code should remain the same (or at least very similar) to the one provided in this notebook.

**Notes:**
1. The `backbone` should have a `forward` method that takes a batch of time series as input and returns the output of the encoder (embeddings). Your encoder must accept samples with the shape `(batch_size, channels, steps)`, where `channels` is the number of channels in the time series and `steps` is the number of time steps. For DAGHAR dataset, `channels=6` and `steps=60`. **Thus, your encoder should accept samples with the shape `(batch_size, 6, 60)`.**



In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from datetime import datetime

import lightning as L
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

from minerva.pipelines.lightning_pipeline import SimpleLightningPipeline
from minerva.models.nets.base import SimpleSupervisedModel

import torchmetrics
from minerva.models.nets.time_series.cnns import CNN_PF_Backbone
from minerva.data.data_modules.har import MultiModalHARSeriesDataModule
from minerva.models.loaders import FromPretrained
from minerva.models.nets.base import SimpleSupervisedModel
from minerva.models.nets.mlp import MLP
from minerva.analysis.metrics.balanced_accuracy import BalancedAccuracy
from minerva.analysis.model_analysis import TSNEAnalysis


/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Dev/minerva/pipelines/base.py:16: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/minerva_ssl_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MLP??

Init signature:
MLP(
    layer_sizes: Sequence[int],
    activation_cls: type = <class 'torch.nn.modules.activation.ReLU'>,
    *args,
    **kwargs,
)
Source:        
class MLP(nn.Sequential):
    """
    A multilayer perceptron (MLP) implemented as a subclass of nn.Sequential.

    This MLP is composed of a sequence of linear layers interleaved with ReLU activation
    functions, except for the final layer which remains purely linear.

    Example
    -------

    >>> mlp = MLP(10, 20, 30, 40)
    >>> print(mlp)
    MLP(
        (0): Linear(in_features=10, out_features=20, bias=True)
        (1): ReLU()
        (2): Linear(in_features=20, out_features=30, bias=True)
        (3): ReLU()
        (4): Linear(in_features=30, out_features=40, bias=True)
    )
    """

    def __init__(
        self,
        layer_sizes: Sequence[int],
        activation_cls: type = nn.ReLU,
        *args,
        **kwargs,
    ):
        """
        Initializes the MLP with specified layer sizes.

        

In [4]:
# Name of the experiment
execution_id = f'run_{datetime.now().strftime("%Y%m%d-%H%M%S")}'
# Directory to save logs
log_dir = f"./logs/{execution_id}" 


print(f"Execution ID: {execution_id}")
print(f"Log dir: {log_dir}")

Execution ID: run_20260205-152539
Log dir: ./logs/run_20260205-152539


## Supervised training of Custom Encoder

In this notebook, we will fine-tune a custom encoder on the DAGHAR dataset. The encoder can be any `torch.nn.Module` that was trained elsewhere and whose checkpoint is available at a `.ckpt` file (saved using `torch.save(model.state_dict())`). This file may contain only the weights from encoder or may contains the weights of the entire model, including the encoder.

### 1. Defining the Data Module

We will use the `MultiModalHARSeriesDataModule` data module to load the DAGHAR dataset for fine-tuning. This data module loads the data in the format required for fine-tuning, which includes sliding window time series data for each sample. Thus, each sample of the dataset will be a 2-element tuple containing the time series (`6x60`, where 6 is the number of features and 60 is the window size) and the corresponding label.

The data module requires the following arguments:
- `data_path`: Path to the directory containing the dataset (change it to use other dataset from DAGHAR).
- `feature_prefix`: The prefix of the columns containing the features. For each prefix, we will create a different channel with all columns that start with the prefix.
- `label`: The name of the column containing the labels.
- `features_as_channels`: If True, for each prefix, we will create a different channel with all columns that start with the prefix. If False, we will concatenate all columns with the same prefix into a single channel (the sample will be a tensor of 1x360 instead of 6x60).
- `cast_to`: The data type to cast the features (float32).
- `batch_size`: The batch size for training.

In [5]:
!ls ../../../../../

figures			 Minerva-Dev   minerva_ssl_env	setup_dev_container.py
install_requirements.sh  Minerva-Exps  README.md	shared_data


In [6]:
data_module = MultiModalHARSeriesDataModule(
    data_path="../../../../../shared_data/daghar/standardized_view/KuHar/",
    feature_prefixes=["accel-x", "accel-y", "accel-z", "gyro-x", "gyro-y", "gyro-z"],
    label="standard activity code",
    features_as_channels=True,
    cast_to="float32",
    batch_size=64,
    num_workers=1,
)

data_module

MultiModalHARSeriesDataModule(data_path=../../../../../shared_data/daghar/standardized_view/KuHar, batch_size=64)

### 2. Defining the Supervised Model

We first going to create the `backbone` of our model, that is our encoder. To do this, we:
1. Copy-and-paste the code of the encoder in the cell below. Should be a `torch.nn.Module` or equivalent. You should copy all code necessary to define the encoder, including imports, class definition, the `forward` method, and any other methods or classes that are necessary to define the encoder.

**NOTE**: 
1. Your encoder **must accept samples with the shape `(batch_size, 6, 60)`**. This is the shape of the samples in the DAGHAR dataset. If your encoder does not accept samples with this shape, you should modify it to accept samples with this shape.

#### Encoder Definition*

In [7]:

from minerva.models.nets.tnc import RnnEncoder,TSEncoder
from minerva.models.nets.mlp import MLP
from minerva.models.nets.base import SimpleSupervisedModel
from minerva.models.nets.time_series.resnet import _ResNet1D, ResNetSEBlock
from minerva.models.nets.time_series.cnns import CNN_PF_Backbone
from minerva.models.nets.time_series.imu_transformer import _IMUTransformerEncoder
from minerva.models.nets.lfr_har_architectures import HARSCnnEncoder

backbone = CNN_PF_Backbone(include_middle=True) #768

# backbone = HARSCnnEncoder(dim=2304,input_channel= 6,inner_conv_output_dim= 1280) #2304

# backbone = _ResNet1D(input_shape= (6, 60),residual_block_cls=ResNetSEBlock) #64

# backbone = RnnEncoder(
#         hidden_size=100,
#         in_channel=6,
#         encoding_size=320,
#         bidirectional=True,
#         num_layers=1,
#         dropout=0,
#         cell_type='GRU'
#         # device='cuda',
#         permute=True
#     ) #320

# backbone = TSEncoder(input_dims=6, output_dims=320, hidden_dims=64, depth=10,permute=True) #320

# backbone = _IMUTransformerEncoder(
#         input_shape= (6, 60),
#         transformer_dim = 64,
#         encode_position = True,
#         nhead= 8,
#         dim_feedforward = 128,
#         transformer_dropout = 0.1,
#         transformer_activation = "gelu",
#         num_encoder_layers = 6,
#     )


#### Defining the head

We will define the head of the model. The head is a simple MLP classifier that receives the embeddings from the encoder and outputs the logits. The head consists of a single hidden layer with 128 units and an output layer with the number of classes in the dataset, that is, 6.
In order to know the input size of the head, we need to calculate the embedding size of the encoder. This is done by passing a dummy input through the encoder and checking the output shape. Let's pick the first batch of the training data and pass it through the encoder to get the embedding size.

In [8]:
# Pega os dataloader de treino
data_module.setup("fit")
train_data_loader = data_module.train_dataloader()

# Obtem o primeiro batch de treino (64 amostras de 6x60)
first_batch = next(iter(train_data_loader))

X, y = first_batch
print(f"O primeiro batch de treino tem shape X={tuple(X.shape)} e y={tuple(y.shape)}")

Using DataLoader with shuffle=True
O primeiro batch de treino tem shape X=(64, 6, 60) e y=(64,)


We made a single forward pass through the encoder with the first batch of the training data and checked the output shape that is `(64, 16, 60)`. The embedding size is `16 x 30 = 480`, that is, the product of the number of channels and the number of steps in the output of the encoder. Thus, the input layer of our MLP head will have 480 units.

In [9]:
embeddings = backbone(X)
# multiplica produtorio de todas as camadas com exceção da de batches
mlp_input_shape = embeddings.shape[1] #* embeddings.shape[2]
print(f"O embedding tem shape {tuple(embeddings.shape)}")
print(f"O embedding final, achatado, tem o shape de {mlp_input_shape}")

O embedding tem shape (64, 64, 12)
O embedding final, achatado, tem o shape de 64


Let's define the head of the model in the cell below. The head will be a simple MLP classifier with:
- A input layer with 480 units (`mlp_input_shape` variable).
- A hidden layer with 128 units.
- An output layer with 6 units, that is the number of classes in the dataset (`num_classes` variable).

In [10]:
num_classes = 6
head = MLP([768, 128, num_classes])
head

MLP(
  (0): Linear(in_features=768, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=6, bias=True)
)

#### Defining the SimpleSupervisedModel

Finally, we use the `SimpleSupervisedModel` class to create the supervised model with the encoder as the backbone and the MLP classifier as the head. The `SimpleSupervisedModel` class requires the following arguments:
- `backbone`: The backbone model (`FromPretrained` model).
- `fc`: The head model (the MLP classifier).
- `loss_fn`: The loss function to use for training (`CrossEntropyLoss`).
- `flatten`: Whether to flatten the input before passing it through the head. Usually, the input is flattened if the backbone outputs a tensor with more than two dimensions.
- `train_metrics`: A dictionary where the keys are the names of the metrics and the values are the functions that calculate the metrics. The metrics function should use `torchmetrics` API to calculate the metrics.
- `val_metrics`: A dictionary where the keys are the names of the metrics and the values are the functions that calculate the metrics. The metrics function should use `torchmetrics` API to calculate the metrics.

In [11]:
model = SimpleSupervisedModel(
    backbone=backbone,
    fc=head,
    loss_fn=torch.nn.CrossEntropyLoss(),
    flatten=True,
    train_metrics={
        "acc": torchmetrics.Accuracy(task="multiclass", num_classes=6),
    },
    val_metrics={
        "acc": torchmetrics.Accuracy(task="multiclass", num_classes=6),
    },
)

model

SimpleSupervisedModel(
  (backbone): CNN_PF_Backbone(
    (first_padder): ZeroPadder2D(pad_at=(3,), padding_size=2)
    (upper_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (lower_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (middle_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (shared_part): Sequential(
      (0): Conv2d(48, 64, kernel_size=(3, 5), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
  )
  (fc): MLP(
    (0): Linear(in_featu

In [12]:
# Option 1: Measure ALL model parameters (including backbone AND head)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total model parameters: {total_params:,}")
print(f"Trainable model parameters: {trainable_params:,}")

# Option 2: Measure components separately (as you intended)
# Backbone parameters
backbone_total = sum(p.numel() for p in model.backbone.parameters())
backbone_trainable = sum(p.numel() for p in model.backbone.parameters() if p.requires_grad)

# Head parameters (use model.fc, not head variable)
head_total = sum(p.numel() for p in model.fc.parameters())
head_trainable = sum(p.numel() for p in model.fc.parameters() if p.requires_grad)

print(f"\nBackbone:")
print(f"  Total parameters: {backbone_total:,}")
print(f"  Trainable parameters: {backbone_trainable:,}")

print(f"\nClassification Head:")
print(f"  Total parameters: {head_total:,}")
print(f"  Trainable parameters: {head_trainable:,}")

# Verify the sum matches Option 1
print(f"\nVerification:")
print(f"Backbone + Head total: {backbone_total + head_total:,}")
print(f"Should match total: {total_params:,}")

from thop import profile
evaluation_data = torch.rand(1000, 6, 60, device='cuda')
model.to('cuda')
macs, params = profile(model, inputs=(evaluation_data,))

print(f"MACs: {macs:,}")
print(f"Parameters: {params:,}")


torch.cuda.is_available()
torch.cuda.device_count()





Total model parameters: 145,830
Trainable model parameters: 145,830

Backbone:
  Total parameters: 46,624
  Trainable parameters: 46,624

Classification Head:
  Total parameters: 99,206
  Trainable parameters: 99,206

Verification:
Backbone + Head total: 145,830
Should match total: 145,830


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
MACs: 1,673,856,000.0
Parameters: 145,830.0


1

### 3. Defining the Pytorch Lightning Trainer Configuration

We will define the PyTorch Lightning Trainer configuration for fine-tuning the model.

The trainer configuration includes the following parameters:
- `max_epochs`: The maximum number of epochs to train the model.
- `accelerator`: The accelerator to use for training: `cpu`, `gpu`, or `tpu`.
- `devices`: The number of accelerators to use for training.
- `logger`: The logger to use for logging the training progress.
- `limit_*_batches`: The number of batches to use for training and validation. **This is useful for debugging and testing the model.**. You should remove these parameters when training the model for real.

In [13]:
## Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/',
    monitor='val_loss',
    mode='min',
    save_last=True
)

## Logger
logger = CSVLogger(save_dir=log_dir, name='cpc-finetune', version=execution_id)

## Trainer
trainer = L.Trainer(
    # Maximum number of epochs to train
    max_epochs=20,
    # Training on GPU
    accelerator="cpu",
    # We will train using 1 gpu
    devices=1,
    # Logger for logging
    logger=logger,
    # List of callbacks
    callbacks=[checkpoint_callback],
    # Only for testing. Remove for production. We will only train using 1 batch of training and validation
    # limit_train_batches=1,
    # limit_val_batches=1,
)

trainer

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/minerva_ssl_env/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:175: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.


### 4. Creating the fine-tuning pipeline (and running the training)

We will create a `SimpleLightningPipeline` to fine-tune the model. This pipeline receives the following arguments:
- `model`: The model to train.
- `trainer`: The PyTorch Lightning Trainer to use for training.
- `log_dir`: The directory to save the logs, checkpoints, and other artifacts of the training process.
- `save_run_status`: If True, save the status of the run to the log directory. This is useful for reprodutibility purposes.
- `seed`: The seed to use for random number generators in PyTorch, NumPy, and other libraries.

This pipeline is optional, as user can simply use `trainer.fit(model, datamodule)` directly to train the model. However, the pipeline provides a more organized way to train the model and save required information for reproducibility.

In [14]:
train_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42
)

Run the pipeline to fine-tune the model on the DAGHAR dataset!

In [15]:
train_pipeline.run(data_module, task="fit")

** Seed set to: 42 **
Pipeline info saved at: /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/logs/run_20260205-152539/run_2026-02-05-15-25-40ff081ec5.yaml


/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/minerva_ssl_env/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:881: Checkpoint directory /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/checkpoints exists and is not empty.


┏━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name     ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ backbone │ CNN_PF_Backbone  │ 46.6 K │ train │     0 │
│ 1 │ fc       │ MLP              │ 99.2 K │ train │     0 │
│ 2 │ loss_fn  │ CrossEntropyLoss │      0 │ train │     0 │
└───┴──────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 145 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 145 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 23                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/minerva_ssl_env/lib/python3.11/sit
e-packages/rich/live.py:260: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

Using DataLoader with shuffle=True

/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/minerva_ssl_env/lib/python3.11/sit
e-packages/lightning/pytorch/trainer/connectors/data_connector.py:434: The 'train_dataloader' does not have many 
workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` 
in the `DataLoader` to improve performance.

/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/minerva_ssl_env/lib/python3.11/sit
e-packages/lightning/pytorch/loops/fit_loop.py:317: The number of training batches (21) is smaller than the logging
interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the 
training epoch.

`Trainer.fit` stopped: `max_epochs=20` reached.


⏱️ fit took 13.63s → saved to /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/logs/run_20260205-152539/timings_fit.csv
Pipeline info saved at: /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/logs/run_20260205-152539/run_2026-02-05-15-25-40ff081ec5.yaml


## Evaluating the Fine-tuned Model

After fine-tuning the encoder on the DAGHAR dataset, we will evaluate the model's performance on the test set of the same dataset. We create a simple evaluation pipeline, that will:
1. Run forward on test set.
2. Calculcate the metrics. This is specified in the `classification_metrics` dictionary, where the keys are the names of the metrics and the values are the functions that calculate the metrics. The metrics function should use `torchmetrics` API to calculate the metrics.
3. Perform model analysis, such as plot t-sne embeddings.

The test pipeline requires the following arguments:
- `model`: The fine-tuned model to evaluate.
- `trainer`: The PyTorch Lightning Trainer configuration.
- `log_dir`: The directory to save the evaluation logs.
- `seed`: The random seed for reproducibility.
- `classification_metrics`: A dictionary where the keys are the names of the metrics and the values are the functions that calculate the metrics. The metrics function should use `torchmetrics` API to calculate the metrics.
- `model_analysis`: A function that performs model analysis, such as plotting t-sne embeddings.

Finally, we run the evaluation pipeline with the test data module to evaluate the model on the test set. We set the `task` parameter of the `run` method to `evaluate` to evaluate the model on the test set.

In [16]:
test_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42,
    classification_metrics={
        "accuracy": torchmetrics.Accuracy(num_classes=6, task="multiclass"),
        "f1": torchmetrics.F1Score(num_classes=6, task="multiclass"),
        "precision": torchmetrics.Precision(num_classes=6, task="multiclass"),
        "recall": torchmetrics.Recall(num_classes=6, task="multiclass"),
        # "balanced_accuracy": BalancedAccuracy(num_classes=6, task="multiclass",adjusted=False), # not used anymore
        "balanced_accuracy": torchmetrics.Accuracy(num_classes=6, task="multiclass"),
    },
    apply_metrics_per_sample=False,
    model_analysis={
        "tsne": TSNEAnalysis(
        text_size=28,
        label_names={
            0: "sit",
            1: "stand",
            2: "walk",
            3: "stair up",
            4: "stair down",
            5: "run",
            6: "stair up and down",
        },
        marker_symbols={
            "sit": "x-open",
            "stand": "cross-open",
            "stair up": "triangle-up-open",
            "stair down": "triangle-down-open",
            "walk": "circle-open",
            "run": "star-open",
        },
        colors={
            "sit": "#1f77b4",
            "stand": "#ff7f0e",
            "walk": "#2ca02c",
            "stair up": "#9467bd",
            "stair down": "#d62728",
            "run": "#8c564b",
        },
        height=1000,
        width=1000,
        legend_title="Activity",
        title=" ",
        output_filename="tnse_analysis.png",
        x_axis_title="1st Component",
        y_axis_title="2nd Component",
    )
}
)






In [17]:
test_pipeline.run(
    data_module, task="evaluate", ckpt_path=checkpoint_callback.best_model_path
)

Restoring states from the checkpoint path at /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/checkpoints/epoch=0-step=21-v2.ckpt
Loaded model weights from the checkpoint at /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/checkpoints/epoch=0-step=21-v2.ckpt


** Seed set to: 42 **
Pipeline info saved at: /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/logs/run_20260205-152539/run_2026-02-05-15-25-549208cccf.yaml
Using DataLoader with shuffle=False
🔍 True labels shape: torch.Size([144])
🔍 Unique true classes: tensor([0, 1, 2, 3, 4, 5])
🔍 True class distribution: tensor([24, 24, 24, 24, 24, 24])
Using DataLoader with shuffle=False


/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/minerva_ssl_env/lib/python3.11/site-packages/lightning/pytorch/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/minerva_ssl_env/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:434: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/minerva_ssl_env/lib/python3.11/sit
e-packages/rich/live.py:260: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



 saving y and yhat 


🔍 Raw predictions shape: torch.Size([144, 6])
Running classification metrics...

📊 Misclassification summary
Total: 67/144 (46.5278%)
Class 0: 0/24 (0.00%)
Class 1: 24/24 (100.00%)
Class 2: 22/24 (91.67%)
Class 3: 0/24 (0.00%)
Class 4: 21/24 (87.50%)
Class 5: 0/24 (0.00%)
🔍 Predicted classes shape: torch.Size([144])
🔍 Unique predicted classes: tensor([0, 2, 3, 4, 5])
🔍 Predicted class distribution: tensor([48,  0,  2, 47,  3, 44])
Metric accuracy: [0.5347222089767456]
Metric f1: [0.5347222089767456]
Metric precision: [0.5347222089767456]
Metric recall: [0.5347222089767456]
Metric balanced_accuracy: [0.5347222089767456]
🔍 Manual accuracy: 0.5347 (77/144)
BalancedAccuracy()


🔍 Balanced Accuracy: 0.5347


Running model analysis...


/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Dev/minerva/analysis/model_analysis.py:196: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  X = torch.tensor(X, device="cpu")


Using DataLoader with shuffle=False
t-SNE PNG saved to /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/logs/run_20260205-152539/tnse_analysis.png
t-SNE HTML saved to /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/logs/run_20260205-152539/tnse_analysis.html
Metrics saved to /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/logs/run_20260205-152539/metrics_2026-02-05-15-25-549208cccf.yaml
⏱️ evaluate took 2.27s → saved to /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/logs/run_20260205-152539/timings_evaluate.csv
Pipeline info saved at: /home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/do

{'classification': {'accuracy': [0.5347222089767456],
  'f1': [0.5347222089767456],
  'precision': [0.5347222089767456],
  'recall': [0.5347222089767456],
  'balanced_accuracy': [0.5347222089767456]},
 'analysis': {'tsne': {'png_path': '/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/logs/run_20260205-152539/tnse_analysis.png',
   'html_path': '/home/gustavo-luz/code/hiaac/paper_access_private_git/Benchmarking_Enconders_SSL/Minerva-Exps/benchmarks/experiments/docs/supervised/logs/run_20260205-152539/tnse_analysis.html'}},
 'misclassification': {'total': {'samples': 144,
   'misclassified': 67,
   'rate': 0.4652777777777778},
  'per_class': {0: {'total': 24,
    'misclassified': 0,
    'misclassification_rate': 0.0},
   1: {'total': 24, 'misclassified': 24, 'misclassification_rate': 1.0},
   2: {'total': 24,
    'misclassified': 22,
    'misclassification_rate': 0.9166666666666666},
   3: {'total': 24, 